In [ ]:
# /wamsi-westport-project-1-1/csiem-data/data-lake/ESA/GlobColor/Transp/Point/CMEMS_transp_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/ESA/GlobColor/Transp/Polygon/CMEMS_transp_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/ESA/GlobColor/Optics/Point/CMEMS_optics_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/ESA/GlobColor/Optics/Polygon/CMEMS_optics_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/ESA/GlobColor/PP/Point/CMEMS_PP_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/ESA/GlobColor/PP/Polygon/CMEMS_PP_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/UKMO/OSTIA/Temperature/Points/CMEMS_SST_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/UKMO/OSTIA/Temperature/Polygon/CMEMS_SST_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/NASA/MODIS/PAR/Points/MODIS_PAR_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/NASA/MODIS/PAR/Polygon/MODIS_PAR_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/NASA/MODIS/PIC/Points/MODIS_PIC_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/NASA/MODIS/PIC/Polygon/MODIS_PIC_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/NASA/MODIS/POC/Points/MODIS_POC_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/NASA/MODIS/POC/Polygon/MODIS_POC_polygon_1.csv

# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_bio/Points/CMEMS_bio_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_bio/Polygon/CMEMS_bio_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_Nut/Points/CMEMS_nut_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_Nut/Polygon/CMEMS_nut_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_optics/Points/CMEMS_optics_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_optics/Polygon/CMEMS_optics_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_co2/Points/CMEMS_co2_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_co2/Polygon/CMEMS_co2_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_car/Points/CMEMS_car_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_car/Polygon/CMEMS_car_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_pft/Points/CMEMS_pft_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/PISCES/Model_pft/Polygon/CMEMS_pft_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/SEAPODYM/Model_PP_ZO/Points/CMEMS_npp_zooc_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/SEAPODYM/Model_PP_ZO/Polygon/CMEMS_npp_zooc_polygon_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/NEMO/Model_salinity/Points/CMEMS_Salt_point_1.csv
# /wamsi-westport-project-1-1/csiem-data/data-lake/MOI/NEMO/Model_salinity/Polygon/CMEMS_Salt_polygon_1.csv